In [ ]:
!pip install keras_tuner --quiet
!pip install tensorflow-io --quiet
# Google colab modules
from google.colab import drive
import sys, importlib

# Mount drive
drive._mount('/content/gdrive', force_remount=True)
ROOT_PATH = './gdrive/MyDrive/#PROJECT/idx/'
sys.path.append(ROOT_PATH)

import coremlv2 as core
core._init_ml()

# Reload coreml
importlib.reload(core)

# Training using whole data: hyperparam from keras tuner

In [ ]:
# Load dataset

# Load model

# Load model_weights

# Recommendation algorithm v1 - model opt: Keras Tuner

In [ ]:
!pip install keras_tuner

In [ ]:
ticker_group = ['BMRI']
shift = 0
interval = 1
recurrent = 120
db_ver = '3'
dataset_ver = '4'
kt_iter = '1'
split = 0.8

import keras_tuner as kt

In [ ]:
train_inputs, train_labels, train_changes, test_inputs, test_labels, test_changes, data_version = core.load_dataset(ticker_group=ticker_group, shift=shift, interval=interval, recurrent=recurrent, db_ver=db_ver, dataset_ver=dataset_ver, split=split, ROOT_PATH=ROOT_PATH)

In [ ]:
tuner = kt.Hyperband(hypermodel=core.model_263_kt, objective='val_loss', max_epochs=100, hyperband_iterations=2, overwrite=True, directory=f'{ROOT_PATH}models/kt/v{kt_iter}/', project_name='_'.join(ticker_group))

In [ ]:
tuner.search_space_summary()

In [ ]:
tuner.search(core.np.nan_to_num(train_inputs, posinf=0.0, neginf=0.0), train_labels, validation_data=(core.np.nan_to_num(test_inputs, posinf=0.0, neginf=0.0), test_labels), epochs=50, verbose=2)

# IDX Explore

In [ ]:
# Load all data
df = core.load_all_data(ROOT_PATH)

In [ ]:
# Model Engineering ####
label_column = 'close_EMA3_G_s2'
label_columns = ['close_EMA3_G_s1','close_EMA10_G_s2','close_EMA3_G_s2','close_EMA10_G_s1']
recurrent = 5
explore_version = 3
epochs = 20
batch_size = 128
train_shuffle = True
checkpoint_dir = core.os.path.join(ROOT_PATH, f'models/explore_idx/v{explore_version}/interim/checkpoint')
model_no = '188'
########################

train_df, test_df, train_mean, train_std = core.split_traintest(df, split=0.8)
train_df_input = core.drop_unallowed_columns(train_df)
test_df_input = core.drop_unallowed_columns(test_df)

input_c = [column for column in train_df_input.columns if column != label_column]

# Resume from latest checkpoint
checkpoint_dir = core.os.path.join(ROOT_PATH, f'models/explore_idx/v{explore_version}/interim/checkpoint/')
model, latest_epoch, latest_label_column, latest_label_index, label_columns = core.resume_from_checkpoint(checkpoint_dir, model_no, recurrent, train_df_input, label_columns)

if (len(label_columns) == 1) & (latest_epoch == epochs):
    sys.exit('Training has been finished.')
else:
    pass

histories = []
first = True
for label_column in label_columns:
    if first:
        current_epochs = epochs - latest_epoch
        first = False
    elif not first:
        current_epochs = epochs
    # Create checkpoint
    checkpoint_path = checkpoint_dir + label_column + 'cp-{epoch:04d}.ckpt'
    cp_callback = core.tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, verbose=1, save_weights_only=True)

    train_dataset = core.TimeseriesGenerator(train_df[input_c], train_df[label_column], length=recurrent, batch_size=batch_size, shuffle=train_shuffle)
    test_dataset = core.TimeseriesGenerator(test_df[input_c], test_df[label_column], length=recurrent, batch_size=batch_size)
    history = model.fit(train_dataset, validation_data=test_dataset, epochs=current_epochs, verbose=1, callbacks=[cp_callback])
    histories.append(history)

    # Save model
    model.save_weights(f'{ROOT_PATH}models/explore_idx/v{explore_version}/interim/weights/{label_column}')
    model.save(f'{ROOT_PATH}models/explore_idx/v{explore_version}/interim/model/{label_column}')

# Save model
model.save_weights(f'{ROOT_PATH}models/explore_idx/v{explore_version}/final/weights/')
model.save(f'{ROOT_PATH}models/explore_idx/v{explore_version}/final/model/')